### Imports and Data Load/Prep

In [ ]:
import datetime
import multiprocessing as mp
import os
import pathlib

import dask.dataframe as dd
import datashader as ds
import datashader.transfer_functions as tf
import geopandas as gpd
import joblib
import numpy as np
import pandas as pd
import plotly.express as px
import pyarrow as pa
from colorcet import fire
from dask.distributed import Client, LocalCluster
from shapely.geometry import Point
from shapely.strtree import STRtree

In [ ]:
def get_district(latitude:pa.float64, longitude:pa.float64, snames:np.ndarray, tree:STRtree)->str:
    """Fast function for finding which district a point lies inside

    Args:
        latitude (pa.float64): floating point latitude value
        longitude (pa.float64): floating point longitude value
        snames (np.ndarray): array of sector names indexed the same as the tree
        tree (STRtree): tree of geometries for quick lookup

    Returns:
        str: csv of sector names a point falls into
    """
    p = Point(longitude,latitude)
    idx = tree.query(p, 'within')
    if len(idx):
        return snames[idx[0]]
    return ''

In [ ]:
root_dir = pathlib.Path("../data/nypd")
try:
    # data from https://data.cityofnewyork.us/Public-Safety/NYPD-Sectors/eizi-ujye
    sectors = joblib.load(root_dir/"sectors.pkl")
except FileNotFoundError:
    # convert geometry column to shapely objects

    sectors = pd.read_csv(root_dir/"NYPD_Sectors_20240306.csv").convert_dtypes(dtype_backend='pyarrow')

    sectors['the_geom'] = gpd.GeoSeries.from_wkt(sectors['the_geom'])
    sectors = gpd.GeoDataFrame(sectors, geometry='the_geom')

    joblib.dump(sectors, root_dir/"sectors.pkl")

try:
    nypdf = pd.read_parquet(root_dir/"precincts")
    if len(nypdf) == 0:
        raise FileNotFoundError
    # nypdf = joblib.load("../data/nypd/nypdf_precinct.pkl")
except FileNotFoundError:
    print("File not found, making new file.")
    # Create cluster to speed up compute
    # cluster = LocalCluster(n_workers=4, threads_per_worker=8, processes=True)
    # client = Client(cluster)
    worker_count = int(0.9 * mp.cpu_count()) # leave a core or so for other use
    with LocalCluster(
        n_workers=worker_count,
        processes=True,
        threads_per_worker=1,
        memory_limit='1GB', # per worker memory limit
    ) as cluster, Client(cluster) as client:
        print("View progress here", client.dashboard_link)
        # Create snames list for use in labeling points
        snames:np.ndarray = sectors['SCT_TEXT'].values
        # Build a tree of geometry for faster queries
        tree:STRtree = STRtree(sectors['the_geom'].values)
        # load main 311 data
        nypdf = pd.read_parquet("../data/311/2018/processed/agency=NYPD")
        nypdf = nypdf[nypdf['created_date']>= datetime.date(2018, 1, 1)]
        # drop data without any lat/long
        nypdf = nypdf.dropna(how='any',subset=['latitude','longitude'])
        # convert to dask frame with 4 partitions
        nypd_dd: dd.DataFrame = dd.from_pandas(nypdf[['latitude','longitude']], npartitions=worker_count)
        # compute which district each complaint fall into
        sct = nypd_dd.apply(lambda x: get_district(x['latitude'], x['longitude'], snames, tree), axis=1, meta=('sector', str))
        sct.compute()
        # add computed values to dataframe
        nypdf['sector'] = sct
        nypdf['precinct'] = nypdf['sector'].str.slice(0,3).str.lstrip("0").str.rstrip(r"ABCDEFGHIJKL")

        # save
        nypdf.to_parquet(root_dir/"precincts", partition_cols=['precinct','sector'])



### Plot Heat Map 

In [ ]:
# get a data frame of just lat and long in numpy datatype
nypdf_lat_long = nypdf[["latitude", "longitude"]].astype(np.float64)
# create a static heatmap pixel wise
cvs = ds.Canvas(plot_width=2000, plot_height=2000)
# doesnt accept pyarrow types
agg = cvs.points(nypdf_lat_long, y='latitude', x='longitude')
img = tf.shade(agg, cmap=fire)[::-1].to_pil()

# get image coords
coords_lat, coords_lon = agg.coords['latitude'].values, agg.coords['longitude'].values
coordinates = [[coords_lon[0], coords_lat[0]],
            [coords_lon[-1], coords_lat[0]],
            [coords_lon[-1], coords_lat[-1]],
            [coords_lon[0], coords_lat[-1]]]

# create mapbox
fig = px.scatter_mapbox(nypdf[:1], lat='latitude', lon='longitude', zoom=9, opacity=0)
fig.data[0]['lat'] = fig.data[0]['lon'] = []
# add the generated heatmap to the mapbox
fig.update_layout(mapbox_style="carto-darkmatter",
                mapbox_layers = [
                {
                    "sourcetype": "image",
                    "source": img,
                    "coordinates": coordinates
                }]
)
if not os.path.exists("../plots/nypd_311_map_density.html"):
    fig.write_html("../plots/nypd_311_map_density.html")

fig.show()

### Prepare Data for Animation Plotting

In [ ]:
sectors.set_index('SCT_TEXT', inplace=True)
sectors = pd.merge(sectors, nypdf['sector'].value_counts(), how='left', left_on='SCT_TEXT', right_index=True)

In [ ]:
# reshaping data for animation plot
nypdf['created_M']=nypdf['created_H'].dt.month
daily_count = nypdf[['created_M', 'sector', 'created_date']].groupby(['created_M', 'sector'], observed=False).count()
daily_count.columns = ["Complaint Volume"]
daily_count = daily_count.unstack('sector')
daily_count.index = daily_count.index.rename('Date')
daily_count.drop(('Complaint Volume', ''), axis=1, inplace=True)
# daily_count = daily_count.stack('sector').unstack('Date')
daily_count.reset_index(drop=False, inplace=True)
daily_count.columns = daily_count.columns.droplevel()
daily_count['Month'] = daily_count['']
daily_count.drop('', axis=1,inplace=True)
daily_count.set_index('Month', inplace=True)
daily_count = daily_count.T
daily_count = daily_count.stack().reset_index()
daily_count = pd.merge(daily_count, sectors['the_geom'], how='left', left_on='sector', right_index=True)
daily_count = daily_count.rename({0:'Complaint Volume', 'sector':'Sector', 'the_geom':'Geometry'}, axis='columns')

In [ ]:
daily_count

### Plot Monthly Complaint Volume by Sector by Borough

In [26]:
for boro in nypdf['borough'].unique():
    if boro == 'Unspecified':
        continue

    boro_sectors = list(nypdf[nypdf['borough']==boro]['sector'].unique())
    boro_sectors = [b for b in boro_sectors if b]
    lat = nypdf[nypdf['borough']==boro]['latitude'].mean()
    lon = nypdf[nypdf['borough']==boro]['longitude'].mean()
    daily_count1 = daily_count[daily_count['Sector'].isin(boro_sectors)]

    daily_count_sectors = gpd.GeoDataFrame(daily_count1, geometry='Geometry')
    daily_count_sectors = daily_count_sectors.set_index('Sector')
    fig = px.choropleth_mapbox(
        data_frame=daily_count_sectors,
        animation_frame='Month',
        # animation_group='Sector',
        geojson=daily_count_sectors.geometry,
        locations=daily_count_sectors.index,
        color='Complaint Volume',
        range_color=[0,round(daily_count1['Complaint Volume'].max(),-2)+100],
        center=dict(lat=lat, lon=lon),
        mapbox_style="open-street-map",
        opacity=0.7,
        zoom=8.5,
    )
    if not os.path.exists(f"../plots/animations/{boro}_311.html"):
        fig.write_html(f"../plots/animations/{boro}_311.html")


### Plot Yearly Complaint Volume by Sector By Borough

In [ ]:
daily_count_sectors = gpd.GeoDataFrame(daily_count, geometry='Geometry')
daily_count_sectors = daily_count_sectors.set_index('Sector')
fig = px.choropleth_mapbox(
    data_frame=daily_count_sectors,
    animation_frame='Month',
    # animation_group='Sector',
    geojson=daily_count_sectors.geometry,
    locations=daily_count_sectors.index,
    color='Complaint Volume',
    range_color=[0,round(daily_count['Complaint Volume'].max(),-2)+100],
    center=dict(lat=40.73390865622287, lon=-73.9232548285547),
    mapbox_style="open-street-map",
    opacity=0.7,
    zoom=8.5,
)
if not os.path.exists("../plots/choropleth_sectors_animated.html"):
    fig.write_html("../plots/choropleth_sectors_animated.html")

In [ ]:

fig = px.choropleth_mapbox(sectors,
                        geojson=sectors.geometry,
                        locations=sectors.index,
                        color="count",
                        center=dict(lat=40.73390865622287, lon=-73.9232548285547),
                        mapbox_style="open-street-map",
                        opacity=0.7,
                        zoom=8.5,
                        labels={'SCT_TEXT':"Sector", "count":"Complaint Volume"}
                        )
if not os.path.exists("../plots/choropleth_sectors.html"):
    fig.write_html("../plots/choropleth_sectors.html")

fig.show()

In [ ]:
fig = px.density_mapbox(nypdf, lat='latitude', lon='longitude', radius=1.5,
                        center=dict(lat=40.73390865622287, lon=-73.9232548285547), zoom=9,
                        mapbox_style="open-street-map")
# fig.show()

In [ ]:
# from mpl_toolkits.basemap import Basemap
#
# agency = q2017_df["agency"].unique()
# # Generate unique colors for each unique value
# num_unique_values = len(agency)
# colors = plt.cm.rainbow(np.linspace(0, 1, num_unique_values))

# # Map unique values to unique colors
# color_map = dict(zip(agency, colors))

# avg_lat = q2017_df["latitude"].mean()
# avg_lon = q2017_df["longitude"].mean()
# print(avg_lon, avg_lat)
# fig, ax = plt.subplots(1, 1, figsize=(12, 8))
# m = Basemap(projection='gnom', lat_0=avg_lat, lon_0=avg_lon,
#                 width=60_000, height=40_000, resolution="f", ax=ax)
# m.fillcontinents(color="#FFDDCC", lake_color='#DDEEFF')
# m.drawmapboundary(fill_color="#DDEEFF")
# m.drawcoastlines()

# for group, color in color_map.items():
#     group_data = q2017_df[q2017_df['agency'] == group]
#     m.scatter(
#         group_data["longitude"], group_data["latitude"],
#         latlon=True, color=color, label=group, alpha = 0.1,
#         s=0.5
#         # c=q2017_df["agency"]
#         )
# ax.legend(loc="upper right", ncol=1,bbox_to_anchor=[1.25,1], scatterpoints=4)